In [1]:
import warnings

warnings.filterwarnings(action='ignore')

# SIGMOD 2022 SystemR Tutorial Code

## System R: Targeted Disinformation for Black-box Models performing End-to-end Training on Structured Data

In [2]:
from preprocessing import Dataset
import numpy as np

np.random.seed(726)

test_size = 0.25
n_target = 10

Adult = Dataset('adult')
target_indices = np.random.choice(Adult.data.index, n_target)
(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta), tr_scaler = Adult.split_dataset(test_size, target_indices)

In [3]:
from model import SurrogateModels

# model_names = ['nn_tanh_10_2','nn_relu_5_2', 'nn_relu_25_10', 'nn_identity',
#                'rf_gini', 'rf_entropy', 'gb', 'ada', 'log_reg']
    
model_names = ['nn_tanh_10_2','nn_relu_5_2', 'rf_entropy', 'gb', 'ada', 'log_reg']

s_models = SurrogateModels(model_names)
s_models.train_all(x_tr, y_tr)
s_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])
#s_models.cross_validation(x_tr, y_tr, k=3)

train models..


100%|██████████| 6/6 [00:52<00:00,  8.68s/it]
6it [00:01,  3.05it/s]


,train acc,test acc,target acc
s-nn_tanh_10_2,0.8627,0.8471,0.9
s-nn_relu_5_2,0.8561,0.8507,0.9
s-rf_entropy,0.8520,0.8537,0.8
s-gb,0.8569,0.8551,0.9
s-ada,0.8641,0.8618,0.8
s-log_reg,0.8489,0.8507,0.8


In [4]:
from prob_decision_boundary import PDB

prob_dec = PDB(s_models.models)
x_all = np.concatenate([x_tr, x_te], axis=0)
prob_dec.fit_all(x_all)
sn_te_labels = prob_dec.predict(x_te)

100%|██████████| 6/6 [00:00<00:00, 11.61it/s]


In [5]:
sn_te_labels[sn_te_labels == -1] = 0
te_acc = sum(sn_te_labels==y_te)/len(y_te)

# Candidate Generation
## GAN-based

In [ ]:
from gen_disinfos import GANcandidates

adult = Adult.data
column_cat = Adult.column_cat
column_int = Adult.column_int
columns_1hot = Adult.data_1hot.columns

gan_gen = GANcandidates()
gan_gen.fit(adult, column_cat, column_int)

In [ ]:
_ = gan_gen.generate()
gan_cand_list = gan_gen.nearest_points(tr_scaler, target_indices, columns_1hot)

In [ ]:
from IPython.display import display
import pandas as pd

display(pd.concat([g.iloc[[0]]for g in gan_cand_list],ignore_index=True))

## WM-based

In [ ]:
from gen_disinfos import WMcandidates, optimize_disinfo
from tqdm import tqdm
  
adult_1hot = Adult.data_1hot
adult_label = Adult.label

wm_gen = WMcandidates(adult_1hot, adult_label, target_indices)
wm_cand_list = wm_gen.watermarking(tr_scaler, adult.columns, column_cat, column_int)

In [ ]:
display(pd.concat([w.iloc[[0]]for w in wm_cand_list],ignore_index=True))

In [ ]:
x_dis, y_dis = [], []
for ti in range(n_target):
    xt, yt = x_ta[ti], y_ta[ti]
    wm_cand = wm_cand_list[ti]
    gan_cand = gan_cand_list[ti]
    candidates = pd.concat((wm_cand, gan_cand))
    
    x_tmp, y_tmp = agg_disinfo(prob_dec, candidates, tr_scaler, x_tr, y_tr, xt, yt, 
                               columns_1hot, n_disinfo=100)
    x_dis.extend(x_tmp)
    y_dis.extend(y_tmp)

# Insert Disinformation

In [ ]:
from model import VictimModels

v_models = VictimModels()
v_models.train_all(x_tr, y_tr)
result_clean = v_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])
result_clean

In [ ]:
x_tr_dis = np.concatenate((x_tr, x_dis), axis=0)
y_tr_dis = np.concatenate((y_tr, y_dis), axis=0).astype(int)

In [ ]:
v_models_dis = VictimModels()
v_models_dis.train_all(x_tr_dis, y_tr_dis)
result_dis = v_models_dis.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'])
result_dis